In [1]:
from cmdstanpy import CmdStanModel
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
import math
import json
from simulator import popn2_simple_data_morgan

In [2]:
model = CmdStanModel(stan_file='two_popn_number.stan')

In [12]:
n=15
N = [7000,7000,7000]
T = 100
m=2
length=1e6
L = [0.5,10000]
d = popn2_simple_data_morgan(N,T,L,m,length,n,42)


In [13]:
sum(d['y'])/len(d['y'])

2.0365591397849463

In [6]:
def int_p_L_divide_l(N,u,v,t1,t2):
    cu = - (1/N + u/50)
    cv = - (1/N + v/50)
    a1 = np.exp(cu*t2)*(cu*t2-1)/cu**2 - np.exp(cu*t1)*(cu*t1-1)/cu**2
    a2 = np.exp(cv*t2)*(cv*t2-1)/cv**2 - np.exp(cv*t1)*(cv*t1-1)/cv**2
    return (1/(50*N))*(a1-a2)

def likelihood(k,N,u,v,t1,t2):
    lamb = int_p_L_divide_l(N,u,v,t1,t2)
    return np.exp(-lamb)*lamb**k/math.factorial(k)

In [23]:
int_p_L_divide_l(14000,0.5,20000,100,10000000)*200

2.0650544178645065

In [13]:
l = 0
for i in d['y']:
    l+=math.log(likelihood(i,14000,0.5,200,0,1000000000))
l

-1755.372578816046

In [18]:
d['N_obs'] = len(d['y'])
d['N'] = [14000]*3
d['length'] = 200

In [19]:
v = model.variational(data=d,elbo_samples=1000,grad_samples=20)
v.variational_params_dict['T']

22:28:23 - cmdstanpy - INFO - Chain [1] start processing
22:28:25 - cmdstanpy - INFO - Chain [1] done processing


86.0717

In [20]:
v

CmdStanVB: model=two_popn_number['method=variational', 'grad_samples=20', 'elbo_samples=1000', 'adapt', 'engaged=1']
 csv_file:
	/var/folders/gm/vzcskl_s1wzfqy1p36xcp32h0000gn/T/tmphuo7a97c/two_popn_numbervvnivkjf/two_popn_number-20241125222823.csv
 output_file:
	/var/folders/gm/vzcskl_s1wzfqy1p36xcp32h0000gn/T/tmphuo7a97c/two_popn_numbervvnivkjf/two_popn_number-20241125222823_0-stdout.txt

In [21]:
model.optimize(data=d)

22:28:38 - cmdstanpy - INFO - Chain [1] start processing
22:28:38 - cmdstanpy - INFO - Chain [1] done processing


CmdStanMLE: model=two_popn_number['method=optimize']
 csv_file:
	/var/folders/gm/vzcskl_s1wzfqy1p36xcp32h0000gn/T/tmphuo7a97c/two_popn_numberkknx6p5k/two_popn_number-20241125222838.csv
 output_file:
	/var/folders/gm/vzcskl_s1wzfqy1p36xcp32h0000gn/T/tmphuo7a97c/two_popn_numberkknx6p5k/two_popn_number-20241125222838_0-stdout.txt

In [22]:
mcmc = model.sample(data=d)
mcmc.summary()

22:28:46 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

22:28:47 - cmdstanpy - INFO - CmdStan done processing.
22:28:47 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: poisson_lpmf: Rate parameter is nan, but must be nonnegative! (in 'two_popn_number.stan', line 44, column 8 to column 74)
	Exception: poisson_lpmf: Rate parameter is nan, but must be nonnegative! (in 'two_popn_number.stan', line 44, column 8 to column 74)
	Exception: gamma_lpdf: Random variable is inf, but must be positive finite! (in 'two_popn_number.stan', line 40, column 4 to column 20)
	Exception: poisson_lpmf: Rate parameter is nan, but must be nonnegative! (in 'two_popn_number.stan', line 44, column 8 to column 74)
Exception: poisson_lpmf: Rate parameter is nan, but must be nonnegative! (in 'two_popn_number.stan', line 44, column 8 to column 74)
	Exception: poisson_lpmf: Rate parameter is nan, but must be nonnegative! (in 'two_popn_number.stan', line 44, column 8 to column 74)
	Exception: gamma_lpdf: Random variable is inf, but must be positive finit

,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat
lp__,-140.9820,0.019433,0.663079,-142.2990,-140.7250,-140.5080,1164.26,588.603,1.00093
T,84.7749,0.169359,6.039000,75.1339,84.9731,94.7568,1271.49,642.816,1.00046
